# ARMA(p,q)

ARMA(p,q):= is a combination of AR(p) ie autocorrection model, a model that predicts future values based on past values and their white noise, and MA(q) ie moving average that predicts future values based on past errors.

## AR(p)

AR(p) intuitivly lets create an AR model first. AR:= a model that aims to express a time series as a function of its past values; $$X_t = \beta_0 + \beta_1 X_{t-1} + \beta_2 X_{t-2} + \cdots + \beta_p X_{t-p} + \varepsilon_t,
\qquad \varepsilon_t \sim \mathcal{N}(0,\sigma^2)$$.
*epsilon is a white noise. 
Stationarity:= a process is weakly stationary iff; $$E(X)=µ, Var(X)=\sigma^2, Cov(X_{t}, X_{t-h})$$ is dependent on h not t. For AR(1) stationary iff |β|<1, for AR(p) stationary iff $$ β(z) = 1 - β1 z - β2 z^2 - β3 z^3 ... =0$$ are not solutions dont lie in the unit circle, i.e. $| \beta_i |<1$.

estimating the coefficients of β we have a few techniques;

### 1. OLS, 
 we can write it into matrix for; $$ Y = \beta X + \varepsilon $$.
 OLS estimator (minimising square error) := $$ \min_{\beta} \ \hat{\varepsilon}'\hat{\varepsilon}
 = (Y - X\beta)'(Y - X\beta)
 = Y'Y - Y'X\beta - \beta'X'Y + \beta'X'X\beta. $$ due to it being a symetric matrix then; $$ \min_{\beta} \ \hat{\varepsilon}'\hat{\varepsilon}=Y'Y -2β'X'Y +β'X'Xβ$$ 
 $$FOC: \frac{\partial}{\partial \beta} = 0
 = -2X'Y + 2X'X\beta$$
 $$ \qquad\Rightarrow\qquad  \widehat{\beta} = (X'X)^{-1}X'Y. 
  $$
  Lets label this (ARE 1) for when we can compare pros; doesnt require stationarity, cons; No autocorrection optimality, endogeniety breaks model.

### 2. Yule Walker,
 need to assume stationary, let; $$ɣ(h)= Cov(X_t, X_{t-h})$$, note; $$β_0 E( X_t X_{t-k} ) = E(β_1 X_{t-1} X_{t-k} + β_2 X_{t-2} X_{t-k} +...+ β_p X_{t-p} X_{t-k} + ℇ_t X_{t-k}) = β_1 E(X_{t-k} X_{t-1}) +...+β_p E(X_{t-p} X_{t-k}), E(ℇ_t X_{t-k})=0$$ as white noise uncorrelated to past values => $$ɣ(k)= β_1 ɣ(k-1) + β_2 ɣ(k-2) +...+ β_p ɣ(k-p) let β = (β_1, β_2,..., β_p)' , ɣ_p = (ɣ(1), ɣ(2),..., ɣ(p))',$$ $$Γ_p = \begin{pmatrix}
\gamma(0) & \gamma(1) & \cdots & \gamma(p-1) \\
\gamma(1) & \gamma(0) & \cdots & \gamma(p-2) \\
\gamma(2) & \gamma(1) & \cdots & \gamma(p-3) \\
\vdots    & \vdots    & \ddots & \vdots      \\
\gamma(p-1) & \gamma(p-2) & \cdots & \gamma(0)
\end{pmatrix} $$you've seen a covarience matrix before.
Then $Γ_p \hat{β} = ɣ_p$ or $ \hat{β} = {Γ_p}^{-1} ɣ_p $




Extra reading: going back to why if its in the roots are in the unit circle its not stationary just because its interesting: lets add a lag opperator L such that $ L X_t = X_{t-1}, L X_{t-1} = X_{t-2}$ or $ L^2 X_t = X_{t-2}$ then; $$ X_t = β_1 X_{t-1} + β_2 X_{t-2} +...+ β_p X_{t-p} +ℇ_t, $$ becomes; $$ X_t = X_t{ β_1 L + β_2 L^2 +...+ L^p} +ℇ_t, $$ or moving to one side $$ ℇ_t = X_t{ 1 - β_1 L - β_2 L^2 -...- L^p}$$, compact this; $$ ℇ_t = β(L) X_t => X_t = β(L)^{-1} ℇ_t,$$ which is an infinite sum that will only converge if past shocks have diminishing effect which wont happen with roots within the unit circle.



In [16]:
#AR(p); autoregressive

from scipy.optimize import minimize
import yfinance as yf #packages
import numpy as np


# Parameters
T = 200 #how many past data points you will use 
p = 2 #amount of beta terms

#Get data
data = yf.download('^GSPC', start='2020-01-01', end='2025-06-15', interval='1d')
X = data['Close'].tail(T).values  # convert to numpy array
X -= X - X.mean()

def sam_AC(X, h):  # Sample autocovariance function
    T = len(X)
    Xbar = np.mean(X)
    cov = 0
    for t in range(h, T):
        cov += (X[t] - Xbar) * (X[t - h] - Xbar)
    return cov / T

def toeplitz(c):
    n = len(c)
    TP = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            TP[i, j] = c[abs(i - j)].item()
    return TP

def YW(X, p):  # Yule-Walker Estimation
    gamma = np.array([sam_AC(X, h) for h in range(p+1)])
    Gamma_p = toeplitz(gamma[:-1])
    gamma_p = gamma[1:]
    beta_hat = np.linalg.solve(Gamma_p, gamma_p)
    
    # Ensure both are 1D arrays before the dot product
    beta_hat = beta_hat.flatten()
    gamma_p = gamma_p.flatten()
    
    sigma2 = gamma[0] - np.dot(beta_hat, gamma_p)
    return beta_hat, sigma2

# Run estimation
beta_hat, sigma2 = YW(X, p)
print("Estimated AR coefficients (beta_hat):", beta_hat)
print("Estimated noise variance (sigma^2):", sigma2)

#check for stationarity 

poly_coefs = np.concatenate(([1], -beta_hat))

# Find roots
roots = np.roots(poly_coefs)

print("Roots of the characteristic polynomial:", roots)
print("Absolute values of roots:", np.abs(roots))

# Stationarity check
if np.all(np.abs(roots) > 1):
    print("The AR process is stationary")
else:
    print("The AR process is NOT stationary")

/var/folders/r9/p1cb21jx0gx9cpxjwsglndy80000gn/T/ipykernel_84322/3363888553.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('^GSPC', start='2020-01-01', end='2025-06-15', interval='1d')
[*********************100%***********************]  1 of 1 completed

Estimated AR coefficients (beta_hat): [ 0.99749373 -0.00250627]
Estimated noise variance (sigma^2): [3.30042991e-26]
Roots of the characteristic polynomial: [0.99497481 0.00251892]
Absolute values of roots: [0.99497481 0.00251892]
The AR process is NOT stationary


## MA(q) ; moving average

An autoregressive process of order q, AR(q), models a time series  $X_t$ - average X,  as a linear function of its past values shocks:

$$ X_t - \mu_t  =  \varepsilon_t  + \theta_1 \varepsilon_{t-1} + \dots + \theta_q \varepsilon_{t-q} , \quad \varepsilon_i \sim \text{i.i.d. } (0, \sigma^2) $$

Essentially this is a linear regression (although we cant use OLS due to the nature of the equation).
In plane terms, the value of a normalised X at time t is equal to the shocks in the past q periods where the older the stock the less impact on time t's value.

Useful fact any AR(p) model can be represented as a MA($\infty$), using this the inverse of a MA(q) given some conditions can be represented as a AR($\infty$):
$$ MA(1), y_t = \varepsilon_t + \theta_1 \varepsilon_{t-1}$$
in its AR form is:
$$ \varepsilon_t = \sum_{j=0}^{\infty} (\theta)^j y_{t-j}, |\theta| < 1$$
for more complex models you may see complex numbers so we will use notation z terms are your q:
$$ \theta(z) = 1 + \theta_1 z + \theta_2 z^2 + \dots + \theta_q z^q $$
roots of this polynomial must satisfy $|z| > 1$. also the sum changes as we cant directly use $\theta$ to:
$$ \varepsilon_t = \sum_{j=0}^{\infty} (\psi)^j y_{t-j}$$
where: $ \psi_j = -(\theta_1 \psi_{j-1} + \theta_2 \psi_{j-2} +\dots + \theta_{\min{j,q}} \psi_{j-q})$, $\psi_0 =1$

so: $ \psi_1 = -\theta_1 $

$\psi_2 = -(\theta_1 \psi_1 + \theta_2) = {\theta_1}^2 - \theta_2 $


In [17]:
q=1
def ma_loglike(params, y, q):
    mu = params[0]
    thetas = params[1:1+q]
    sigma = params[-1]

    T = len(y)
    u = np.zeros(T)

    # recursively compute innovations
    for t in range(q, T):
        u[t] = y[t] - mu - np.dot(thetas, u[t-q:t][::-1])

    # Gaussian likelihood
    ll = -0.5 * np.sum(np.log(2*np.pi*sigma**2) + (u**2)/(sigma**2))
    return -ll  
def fit_ma(y, q):
    init = np.zeros(q+2)  # μ, θ₁…θ_q, σ
    init[-1] = np.std(y)  # initial σ
    out = minimize(ma_loglike, init, args=(y, q))
    return out.x
theta_hat = fit_ma(X, q)
print("Estimated MA coefficients (mean (~0), theta_hats, variance):", theta_hat)

[2.63081410e-16 1.51996030e-12 5.12589897e-05]


/var/folders/r9/p1cb21jx0gx9cpxjwsglndy80000gn/T/ipykernel_84322/2151775836.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[t] = y[t] - mu - np.dot(thetas, u[t-q:t][::-1])
/var/folders/r9/p1cb21jx0gx9cpxjwsglndy80000gn/T/ipykernel_84322/2151775836.py:15: RuntimeWarning: overflow encountered in square
  ll = -0.5 * np.sum(np.log(2*np.pi*sigma**2) + (u**2)/(sigma**2))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
/var/folders/r9/p1cb21jx0gx9cpxjwsglndy80000gn/T/ipykernel_84322/2151775836.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performin

## combining them:
we substitute the equation for MA(q) with the error as the dependent term into AR(p) to get:
$$ X_t = X_{t-1} \beta_{1} + X_{2} \beta_{t-2} + \cdots + X_{t-p} \beta_{p} 
+u_t + \phi_1 u_{t-1} + \phi_2 u_{t-2} + \dots + \phi_q X_{t-q} $$